# Slow Motion with Super SloMo

This notebook uses [Super SloMo](https://arxiv.org/abs/1712.00080) from the open source project [avinashpaliwal/Super-SloMo](https://github.com/avinashpaliwal/Super-SloMo) to slow down a given video.

This is a modification of [this Colab Notebook](https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/SuperSloMo.ipynb#scrollTo=P7eRRjlYaV1s) by styler00dollar aka "sudo rm -rf / --no-preserve-root#8353" on discord.

This version:
- allows to use Google Drive for own videos
- uses CRF inside the ffmpeg command for better space usage
- custom ffmpeg command possible
- includes experemental audio support
- removes .mkv input restriction and supports different filetypes

May be implemented:
- different output format

Interesting things:
- Can do 1080p without crashing (Dain can only do ~900p with 16GB VRAM)
- 1080p 6x works with Super-Slomo

Simple Tutorial:
- Run cells with these play-buttons that are visible on the left side of the code/text. ```[ ]``` indicate a play-button.

# Check GPU

In [ ]:
!nvidia-smi

In [ ]:
#@markdown # Install avinashpaliwal/Super-SloMo
import os
from os.path import exists, join, basename, splitext, dirname

git_repo_url = 'https://github.com/styler00dollar/Colab-Super-SloMo'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !pip install -q youtube-dl
  ffmpeg_path = !which ffmpeg
  ffmpeg_path = dirname(ffmpeg_path[0])
  
import sys
sys.path.append(project_name)
from IPython.display import YouTubeVideo

# Download pre-trained Model
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
pretrained_model = 'SuperSloMo.ckpt'
if not exists(pretrained_model):
  download_from_google_drive('1IvobLDbRiBgZr3ryCRrWL8xDbMZ-KnpF', pretrained_model)

## Super SloMo on a Youtube Video

In [ ]:
#@markdown #### Example URL: https://www.youtube.com/watch?v=P3lXKxOkxbg
YOUTUBE_ID = 'P3lXKxOkxbg' #@param{type:"string"}
YouTubeVideo(YOUTUBE_ID)

Info:
0 fps means that the video path is wrong or you need to wait a bit for Google Drive to sync and try again.

In [ ]:
%cd /content/
!rm -df youtube.mp4
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Detecting FPS of input file.
import os
import cv2
cap = cv2.VideoCapture('/content/youtube.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)

# Configure
SLOW_MOTION_FACTOR = 3 #@param{type:"number"}
# You can change the final FPS manually
TARGET_FPS = fps*FPS_FACTOR
#TARGET_FPS = 90
print("Target FPS")
print(TARGET_FPS)

In [ ]:
#@markdown # Creating video with sound
!python /content/Colab-Super-SloMo/video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint /content/SuperSloMo.ckpt --video /content/youtube.mp4 --sf {SLOW_MOTION_FACTOR} --fps {TARGET_FPS} --output /content/output.mp4
!youtube-dl -x --audio-format aac https://www.youtube.com/watch?v=$YOUTUBE_ID --output /content/output-audio.aac

# Deleting old video, if it exists
if os.path.exists("/content/output.mp4"):
    os.remove("/content/output.mp4")

# You can change these ffmpeg parameter
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '/content/Colab-Super-SloMo/tmp/*.png' -i /content/output-audio.aac -shortest -crf 18 /content/output.mp4

In [ ]:
#@markdown # Creating video without sound
# Deleting old video, if it exists
if os.path.exists("/content/output.mp4"):
    os.remove("/content/output.mp4")

!python /content/Colab-Super-SloMo/video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint /content/SuperSloMo.ckpt --video /content/youtube.mp4 --sf {SLOW_MOTION_FACTOR} --fps {TARGET_FPS} --output /content/output.mkv

# You can change these ffmpeg parameter
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '/content/Colab-Super-SloMo/tmp/*.png' -crf 18 /content/output.mp4

Now you can playback the video with the last cell or copy the video back to Google Drive

In [ ]:
#@markdown # [Optional] Copy video result to ```"Google Drive/output.mp4"```
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

# Copy video back to Google Drive
!cp /content/output.mp4 "/content/drive/My Drive/output.mp4"

## Super SloMo on a Google Drive Video

The default input path is:
```"Google Drive/input.mp4"```. You can change the path if you want. Just change the file extention if you got a different format.

In [ ]:
#@markdown ## Mount Google Drive and configure paths
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

# Configuration. "My Drive" represents your Google Drive.
# Input file:
INPUT_FILEPATH = "/content/drive/My Drive/input.mp4" #@param{type:"string"}
# Output file path. MP4 is recommended. Another extention will need further code-changes.
OUTPUT_FILE_PATH = "/content/drive/My Drive/output.mp4" #@param{type:"string"}

Info:
0 fps means that the video path is wrong or you need to wait a bit for Google Drive to sync and try again.

In [ ]:
#@markdown ## [Experimental] Create Video with sound
# Detecting FPS of input file.
import os
import cv2
cap = cv2.VideoCapture(f'{INPUT_FILEPATH}')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)
# Configure
#@markdown ## Configuration
SLOW_MOTION_FACTOR = 3 #@param{type:"number"}
FPS_FACTOR = 3 #@param{type:"number"}
# You can change the final FPS manually
TARGET_FPS = fps*FPS_FACTOR
#TARGET_FPS = 90
print("Target FPS")
print(TARGET_FPS)
# Copy video from Google Drive
file_extention = os.path.splitext(INPUT_FILEPATH)[1]
!cp '{INPUT_FILEPATH}' /content/input{file_extention}
!python /content/Colab-Super-SloMo/video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint /content/SuperSloMo.ckpt --video /content/input{file_extention} --sf {SLOW_MOTION_FACTOR}
%shell ffmpeg -i /content/input{file_extention} -acodec copy /content/output-audio.aac
# Deleting old video, if it exists
if os.path.exists("/content/output.mp4"):
    os.remove("/content/output.mp4")
# You can change these ffmpeg parameter
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '/content/Colab-Super-SloMo/tmp/*.png' -i /content/output-audio.aac -shortest -crf 18 /content/output.mp4
# Copy video back to Google Drive
!cp /content/output.mp4 '{OUTPUT_FILE_PATH}'

In [ ]:
#@markdown ## Create video without sound
# Detecting FPS of input file.
import os
import cv2
cap = cv2.VideoCapture(f'{INPUT_FILEPATH}')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)
#@markdown ## Configuration
SLOW_MOTION_FACTOR = 3 #@param{type:"number"}
FPS_FACTOR = 3 #@param{type:"number"}
# You can change the final FPS manually
TARGET_FPS = fps*FPS_FACTOR
#TARGET_FPS = 90
print("Target FPS")
print(TARGET_FPS)
# Copy video from Google Drive
file_extention = os.path.splitext(INPUT_FILEPATH)[1]
!cp '{INPUT_FILEPATH}' /content/input{file_extention}
!cd '{project_name}' && python video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint ../{pretrained_model} --video /content/input{file_extention} --sf {SLOW_MOTION_FACTOR}
# Deleting old video, if it exists
if os.path.exists("/content/output.mp4"):
    os.remove("/content/output.mp4")
# You can change these ffmpeg parameter
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '/content/Colab-Super-SloMo/tmp/*.png' -crf 18 /content/output.mp4
# Copy video back to Google Drive
!cp /content/output.mp4 '{OUTPUT_FILE_PATH}'

In [ ]:
#@markdown ## [Experimental] Create video with sound and removed duplicate frames
import cv2
file_extention = os.path.splitext(INPUT_FILEPATH)[1]
!cp '{INPUT_FILEPATH}' /content/input{file_extention}
# Get amount frames
cap = cv2.VideoCapture("/content/input.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Detecting FPS of input file.
import os
import cv2
cap = cv2.VideoCapture('/content/input.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)
# Deleting old video, if it exists
if os.path.exists("/content/output.mp4"):
    os.remove("/content/output.mp4")
# Configure
SLOW_MOTION_FACTOR = 6 #@param{type:"number"}
FPS_FACTOR = 6 #@param{type:"number"}
TARGET_FPS = fps*FPS_FACTOR
!python /content/Colab-Super-SloMo/video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint /content/SuperSloMo.ckpt --video /content/input.mp4 --sf {SLOW_MOTION_FACTOR} --remove_duplicate True
%shell ffmpeg -i /content/input{file_extention} -acodec copy /content/output-audio.aac
amount_files_created = len(os.listdir(('/content/Colab-Super-SloMo/tmp')))
# You can change these ffmpeg parameter
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '/content/Colab-Super-SloMo/tmp/*.png' -i /content/output-audio.aac -shortest -crf 18 /content/output.mp4
# Copy video back to Google Drive
!cp /content/output.mp4 '{OUTPUT_FILE_PATH}'

In [ ]:
#@markdown ## Create video without sound and removed duplicate frames
import cv2
file_extention = os.path.splitext(INPUT_FILEPATH)[1]
!cp '{INPUT_FILEPATH}' /content/input{file_extention}
# Get amount frames
cap = cv2.VideoCapture("/content/input.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Detecting FPS of input file.
import os
import cv2
cap = cv2.VideoCapture('/content/input.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)
# Deleting old video, if it exists
if os.path.exists("/content/output.mp4"):
    os.remove("/content/output.mp4")
# Configure
SLOW_MOTION_FACTOR = 6 #@param{type:"number"}
FPS_FACTOR = 6 #@param{type:"number"}
TARGET_FPS = fps*FPS_FACTOR
!python /content/Colab-Super-SloMo/video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint /content/SuperSloMo.ckpt --video /content/input.mp4 --sf {SLOW_MOTION_FACTOR} --remove_duplicate True
amount_files_created = len(os.listdir(('/content/Colab-Super-SloMo/tmp')))
# You can change these ffmpeg parameter
%shell ffmpeg -y -r {amount_files_created/(length/fps)} -f image2 -pattern_type glob -i '/content/Colab-Super-SloMo/tmp/*.png' -crf 18 /content/output.mp4
# Copy video back to Google Drive
!cp /content/output.mp4 '{OUTPUT_FILE_PATH}'

In [ ]:
#@markdown ## Preview the result within Colab
#@markdown #### Don't try this with big files. It will crash Colab. Small files like 10mb are ok.
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('/content/output.mp4', width=960, height=720)